# Task 2: LDA Topic Model
***

We create a simple, out-of-the-box Latent Dirichlet Allocation Topic Model with visualization.

In [15]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import pandas as pd
import numpy as np

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

from sklearn.decomposition import LatentDirichletAllocation

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.feature_selection import SelectKBest, chi2

from sklearn.pipeline import Pipeline, FeatureUnion

In [5]:
df = pd.read_csv('cleaned_jels.csv', encoding='utf-8-sig', low_memory=False)
df.drop('0', axis=1, inplace=True)
df

,Title,Publisher,Abstract,Keywords,Jel Codes,Author1,Author2,Author3,Author4,Author5,...,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80
0,technical efficiency of hungarian farms before...,european association of agricultural economists,hungary is one of the ten countries that have ...,"[agribusiness, production economics]",NaN,"zoltán bakucs, lajos","fertő, imre","fogarasi, józsef",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,exploring the flexibility of polish family far...,european association of agricultural economists,no abstract is available for this item,"[farm management, productivity analysis]",NaN,"pieniadz, agata","renner, swetlana","petrick, martin",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,regional clusters in a function of rural devel...,european association of agricultural economists,as a theoretical concept rural development bas...,[community rural urban development],NaN,"sudarić, tihana","zmaić, krunoslav","petrač, božidar",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,the extended metropolitan area in a new member...,european association of agricultural economists,no abstract is available for this item,"[agricultural and food policy, community rural...",NaN,"zolin, m. bruna",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,infrastructural capacity of family owned holdi...,european association of agricultural economists,no abstract is available for this item,"[community rural urban development, land econo...",NaN,"Živković, dragić","dimitrijević, bojan","jelić, sreten","rajić, zoran",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,comparative research of food consumption in se...,european association of agricultural economists,no abstract is available for this item,[food consumption nutrition food safety],NaN,"Мilanović, milan r.","Đorović, milutin","stevanović, simo",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,targeting agricultural and rural development m...,european association of agricultural economists,no abstract is available for this item,"[agricultural and food policy, community rural...",NaN,"segré, andrea","rakić, renata","rokvić, gordana","vittuari, matteo",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,needs assessment analysis of small rural house...,european association of agricultural economists,no abstract is available for this item,[consumer household economics],NaN,"bogdanov, natalija","moslavac, nenad",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,designing a rural development strategy for ser...,european association of agricultural economists,no abstract is available for this item,[community rural urban development],NaN,"cochrane, nancy","reed, michael","jovanović, zlatko",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,rural development and the heritage of chayanov...,european association of agricultural economists,no abstract is available for this item,"[community rural urban development, labor and ...",NaN,"nuppenau, ernst-august",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df = df[:100]
df

,Title,Publisher,Abstract,Keywords,Jel Codes,Author1,Author2,Author3,Author4,Author5,...,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80
0,technical efficiency of hungarian farms before...,european association of agricultural economists,hungary is one of the ten countries that have ...,"[agribusiness, production economics]",NaN,"zoltán bakucs, lajos","fertő, imre","fogarasi, józsef",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,exploring the flexibility of polish family far...,european association of agricultural economists,no abstract is available for this item,"[farm management, productivity analysis]",NaN,"pieniadz, agata","renner, swetlana","petrick, martin",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,regional clusters in a function of rural devel...,european association of agricultural economists,as a theoretical concept rural development bas...,[community rural urban development],NaN,"sudarić, tihana","zmaić, krunoslav","petrač, božidar",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,the extended metropolitan area in a new member...,european association of agricultural economists,no abstract is available for this item,"[agricultural and food policy, community rural...",NaN,"zolin, m. bruna",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,infrastructural capacity of family owned holdi...,european association of agricultural economists,no abstract is available for this item,"[community rural urban development, land econo...",NaN,"Živković, dragić","dimitrijević, bojan","jelić, sreten","rajić, zoran",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,comparative research of food consumption in se...,european association of agricultural economists,no abstract is available for this item,[food consumption nutrition food safety],NaN,"Мilanović, milan r.","Đorović, milutin","stevanović, simo",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,targeting agricultural and rural development m...,european association of agricultural economists,no abstract is available for this item,"[agricultural and food policy, community rural...",NaN,"segré, andrea","rakić, renata","rokvić, gordana","vittuari, matteo",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,needs assessment analysis of small rural house...,european association of agricultural economists,no abstract is available for this item,[consumer household economics],NaN,"bogdanov, natalija","moslavac, nenad",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,designing a rural development strategy for ser...,european association of agricultural economists,no abstract is available for this item,[community rural urban development],NaN,"cochrane, nancy","reed, michael","jovanović, zlatko",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,rural development and the heritage of chayanov...,european association of agricultural economists,no abstract is available for this item,"[community rural urban development, labor and ...",NaN,"nuppenau, ernst-august",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


>We need to lemmatize all words to reduce dimensionality: jogging, jogged, jogs -> jog

In [17]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_tag(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    elif pos_tag.startswith('S'):
        return wordnet.ADJ_SAT
    else:
        return None


class Lemmatizer_CountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(Lemmatizer_CountVectorizer, self).build_analyzer()
        return lambda doc:([lemmatizer.lemmatize(word[0], get_wordnet_tag(word[1])) for word in pos_tag(analyzer(doc)) 
                            if get_wordnet_tag(word[1]) is not None])

In [24]:
count_vectorizer = Lemmatizer_CountVectorizer(stop_words='english', lowercase=True, ngram_range=(2,3), min_df=5)
dtm_cv = count_vectorizer.fit_transform(df['Title'] + ' ' + df['Abstract'])

TypeError: fit() missing 1 required positional argument: 'y'

>We use a Latent Dirichlet Allocation Model. 

In [26]:
lda_cv = LatentDirichletAllocation(n_topics=10, random_state=0)
lda_cv.fit(dtm_cv)

TypeError: float() argument must be a string or a number, not 'SelectKBest'

In [17]:
visual = pyLDAvis.sklearn.prepare(lda_cv, dtm_cv, count_vectorizer, mds='mmds')

In [18]:
visual

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
8      13.362165        1       1 -0.199031  0.414573
0      10.987301        1       2  0.143915 -0.402714
1      10.786657        1       3  0.209151  0.404629
2      10.496868        1       4 -0.215323 -0.376068
6      10.234135        1       5 -0.353907  0.148852
5       9.920298        1       6  0.068241  0.173090
3       9.113952        1       7  0.369581 -0.183689
9       8.908012        1       8 -0.384757 -0.116917
4       8.096928        1       9  0.387343  0.091642
7       8.093684        1      10 -0.025212 -0.153400, topic_info=       Category          Freq                       Term         Total  \
term                                                                     
651     Default  22612.000000         abstract available  22612.000000   
652     Default  22582.000000    abstract available item  22582.000000   
21104   Default  22582.000000             available item  22582.000000   
194118  Default   5504.000000            monetary policy   5504.000000   
106643  Default   4731.000000              exchange rate   4731.000000   
94540   Default   4684.000000                      en el   4684.000000   
94762   Default   4239.000000                      en la   4239.000000   
84316   Default   3508.000000            economic growth   3508.000000   
175898  Default   3760.000000                   long run   3760.000000   
40299   Default   2714.000000             climate change   2714.000000   
261150  Default   3008.000000                rica latina   3008.000000   
166051  Default   2583.000000               labor market   2583.000000   
73178   Default   3329.000000       developing countries   3329.000000   
208965  Default   2606.000000                 panel data   2606.000000   
309102  Default   3536.000000              united states   3536.000000   
219438  Default   2095.000000                  pol ticas   2095.000000   
116718  Default   1991.000000           financial crisis   1991.000000   
208640  Default   1978.000000                     pa ses   1978.000000   
34807   Default   1926.000000               central bank   1926.000000   
139509  Default   2124.000000              human capital   2124.000000   
168793  Default   1684.000000              latin america   1684.000000   
83620   Default   1992.000000                  econ mico   1992.000000   
94946   Default   1683.000000                     en los   1683.000000   
28824   Default   1693.000000      borrowed version item   1693.000000   
655     Default   1693.000000  abstract borrowed version   1693.000000   
317400  Default   1693.000000               version item   1693.000000   
654     Default   1693.000000          abstract borrowed   1693.000000   
28823   Default   1693.000000           borrowed version   1693.000000   
301006  Default   2005.000000                time series   2005.000000   
219406  Default   1703.000000                   pol tica   1703.000000   
...         ...           ...                        ...           ...   
228208  Topic10    618.794065              price changes    619.659918   
293831  Topic10    598.351172    sustainable development    599.217208   
295825  Topic10    558.776518                   tax rate    559.642509   
181004  Topic10    549.301208       management practices    550.167223   
294842  Topic10    523.060218              taken account    523.926293   
116943  Topic10    516.342487           financial market    517.208540   
289901  Topic10    515.740047         sub saharan africa    516.605848   
265367  Topic10    515.162411             saharan africa    516.028212   
98976   Topic10    512.134056           error correction    512.999837   
254118  Topic10    508.238802           renewable energy    509.104786   
54513   Topic10    489.949325                 costa rica    490.816071   
92590   Topic10    432.021561           emerging mark

In [19]:
pyLDAvis.save_html(visual, 'workingpapers_topics.html')